In [ ]:
%matplotlib inline

import sys
sys.path.append('../python_packages_static')
import geopandas as gp
import flopy as fp
import mfsetup
import os
import numpy as np
import matplotlib.pyplot as plt
import yaml
import pyproj

## Using [modflow-setup](https://github.com/aleaf/modflow-setup/tree/develop/mfsetup) it is possible to build only the discretization package to evaluate grid options.

#### get base dir - need this because MODFLOW setup leaves us in the model directory

In [ ]:
basedir = os.getcwd()
basedir

## first get bounding box for the model domain

In [ ]:
#domain = gp.read_file('../source_data/shapefiles/Extents/Model_Extent_HUC12.shp')

domain = gp.read_file('../source_data/shapefiles/rockRiver/rockRiver.shp')

modelName = 'rockRiver'

print(domain.columns)

### get the CRS - we'll need this later on

In [ ]:
#epsg = pyproj.CRS(domain2.crs).to_epsg()
print(domain.crs.axis_info[0].unit_name)

#modflow output indicates meters... but shapefile still shows feet... something is screwy

#testing for eventual spyder porting
plt.plot(*domain.geometry[0].exterior.xy)
plt.show() #this is fine

bounds = domain.geometry[0].bounds

tot_xdist, tot_ydist = bounds[2]+1000-bounds[0], bounds[3]+1000-bounds[1]
print (tot_xdist,tot_ydist)

### set grid spacing and set up grid

In [ ]:
dx = 2500 #feet
dy = dx

xcells = int(np.ceil(tot_xdist/dx))
ycells = int(np.ceil(tot_ydist/dy))

print(xcells,ycells, xcells*ycells)

## Read in the file `example.yml` and modify

In [ ]:
with open('example_rockRiver.yml', 'r') as ifp:
    inyml = yaml.load(ifp, Loader=yaml.FullLoader)
    
inyml['simulation']['sim_ws'] = 'tmp/{0}'.format(modelName)
inyml['model']['simulation'] = modelName
inyml['model']['modelname'] = modelName
   
inyml['setup_grid']['xoff'] = bounds[0]
inyml['setup_grid']['yoff'] = bounds[1]

inyml['dis']['griddata']['delr'] = dy
inyml['dis']['griddata']['delc'] = dx

inyml['dis']['dimensions']['nlay'] = 10
inyml['dis']['dimensions']['nrow'] = ycells
inyml['dis']['dimensions']['ncol'] = xcells

with open('rockRiver.yml', 'w') as ofp:
    yaml.dump(inyml, ofp)

### now try to make just the DIS package

In [ ]:
m = mfsetup.MF6model.setup_from_yaml('rockRiver.yml') #modified discretization.py to autopopulate active cells in new layers

In [ ]:
arr = np.concatenate([m.dis.idomain.array[0] for x in range(10)])
arr = np.reshape(arr, (10,263,270))

m.dis.idomain = arr

m.write()

#type(m)


### We can export the grid information as a shapefile to evaluate
NOTE: Errors indicating "No internet connection or epsg code ..." can be safely disregarded as warnings. They are due to restrictive network security preventing access to spatialreference.org but do not impact behavior of these notebooks.

In [ ]:
m.dis.export('rockRiver{}f.dis.shp'.format(dx))

In [ ]:
os.chdir(basedir)
